<h1 style="color: #1f77b4; font-size: 50px; text-align: center;">
Acquisition des données : archive météorologique historique parisienne
</h1>

# Contexte et objectifs du projet

Ce notebook initie une étude académique portant sur la prévision de la température de l’air à 2 mètres ($temperature_2m$) à Paris, en France.

L’objectif principal de ce notebook spécifique est d’acquérir un jeu de données météorologiques multivarié, de haute fidélité et à résolution horaire. Le périmètre temporel du jeu de données couvre une période multi-décennale (de 2000 à aujourd’hui), ce qui est essentiel pour capturer une large variété de régimes climatologiques, de cycles saisonniers et d’événements météorologiques extrêmes ou atypiques. 

Les données ont été choisies afin de disposer de séries météorologiques fiables, multivariées et à haute fréquence temporelle pour la ville de Paris. Après avoir exploré plusieurs sources et API, les données horaires (pas de temps de 1h) issues de Open-Meteo ont été retenues pour leur qualité scientifique, leur cohérence physique et leur accessibilité open-source, en adéquation avec les objectifs de modélisation fine et reproductible du projet. La méthodologie repose donc sur l’utilisation de l’API Open-Meteo Archive.

Ce notebook constitue l’étape fondatrice de la collecte de données pour l’ensemble du projet. Le format Jupyter Notebook, ici comme pour les autres notebooks, a été choisi de manière délibérée afin de respecter les principes de la recherche FAIR (Findable, Accessible, Interoperable, Reproducible). Chaque étape est explicitement documentée et exécutée par du code afin de garantir une transparence totale et une reproductibilité complète.

In [ ]:
# Initialisation de l'environnement
import os
import requests
import pandas as pd
from datetime import datetime

In [ ]:
# Définition Géospatiale 
latitude = 48.8534
longitude = 2.3488

💡 Fun fact : ces coordonnées ne représentent pas un « centre-ville » arbitraire, mais ont été spécifiquement choisies pour correspondre à l’emplacement de l’Observatoire de Paris.
Ce site est l’un des plus anciens observatoires astronomiques au monde et constitue depuis des siècles un point central des relevés météorologiques parisiens, ce qui en fait un point d’ancrage scientifiquement et historiquement pertinent pour notre étude.

In [ ]:
# Définition temporelle
end_date = "2025-10-25" # Une end_date dynamique (c’est-à-dire « aujourd’hui ») n’est pas adaptée à un travail de recherche académique,
                        # car elle rend le jeu de données non statique et compromet la reproductibilité
                        # C’est pourquoi nous avons choisi de définir une end_date fixe pour notre étude
start_date = "2000-01-01"

In [ ]:
url = "https://archive-api.open-meteo.com/v1/archive" # Source des données
params = {
    "latitude" : latitude,
    "longitude" : longitude, 
    "start_date" : start_date,
    "end_date" : end_date,
    "hourly" : "temperature_2m,relative_humidity_2m,apparent_temperature,dew_point_2m,precipitation,rain,snowfall,snow_depth,wind_speed_100m,wind_speed_10m,wind_direction_10m,wind_direction_100m,wind_gusts_10m,weather_code,pressure_msl,surface_pressure,cloud_cover,cloud_cover_low,cloud_cover_mid,cloud_cover_high,et0_fao_evapotranspiration,vapour_pressure_deficit,soil_temperature_0_to_7cm,soil_temperature_7_to_28cm,soil_temperature_28_to_100cm,soil_temperature_100_to_255cm,soil_moisture_0_to_7cm,soil_moisture_7_to_28cm,soil_moisture_28_to_100cm,soil_moisture_100_to_255cm,wet_bulb_temperature_2m,boundary_layer_height,total_column_integrated_water_vapour,is_day,sunshine_duration",
    "timezone": "Europe/Berlin",
    "timezone_abbreviation": "CEST"
}    # L’élément « hourly » fait référence à la sélection des variables (Feature Selection), c’est-à-dire à la liste des variables météorologiques à récupérer.

In [5]:
response = requests.get(url, params = params)
Data = response.json()

In [ ]:
# Structuration des données : du JSON au DataFrame de séries temporelles
df = pd.DataFrame(Data["hourly"])
df['time'] = pd.to_datetime(df['time']) 

In [7]:
df.head()

,time,temperature_2m,relative_humidity_2m,apparent_temperature,dew_point_2m,precipitation,rain,snowfall,snow_depth,wind_speed_100m,...,soil_temperature_100_to_255cm,soil_moisture_0_to_7cm,soil_moisture_7_to_28cm,soil_moisture_28_to_100cm,soil_moisture_100_to_255cm,wet_bulb_temperature_2m,boundary_layer_height,total_column_integrated_water_vapour,is_day,sunshine_duration
0,2000-01-01 00:00:00,6.0,97,4.0,5.5,0.0,0.0,0.0,0.01,14.0,...,10.8,0.414,0.419,0.417,0.359,5.6,115.0,15.6,0,0.0
1,2000-01-01 01:00:00,6.0,96,3.8,5.4,0.0,0.0,0.0,0.01,14.9,...,10.8,0.414,0.419,0.417,0.359,5.5,100.0,16.5,0,0.0
2,2000-01-01 02:00:00,5.9,96,3.8,5.3,0.0,0.0,0.0,0.01,14.7,...,10.8,0.414,0.418,0.417,0.359,5.4,105.0,16.7,0,0.0
3,2000-01-01 03:00:00,6.0,96,3.8,5.3,0.0,0.0,0.0,0.01,14.6,...,10.8,0.414,0.418,0.417,0.359,5.5,110.0,16.9,0,0.0
4,2000-01-01 04:00:00,6.1,96,3.9,5.5,0.0,0.0,0.0,0.01,15.5,...,10.7,0.413,0.418,0.417,0.359,5.6,130.0,17.3,0,0.0


In [ ]:
# Persistance des données : export vers un fichier CSV
data_folder = "../data"
os.makedirs(data_folder, exist_ok=True)

data_path = os.path.join(data_folder, "hourly_data.csv")
df.to_csv(data_path, index=False)

print(f"Data saved to {data_path}")

Data saved to ../data/hourly_data.csv
